# Generating Shapley plots for the saved model 

# Predicting diseases - Aggregation vs Mitochondrial toxicity with PowerTransformer

Loading the saved training and test data.
The shap values generated are being saved in the path assigned to the variable 'path_res' 

In [ ]:

import pandas as pd
import sklearn

In [ ]:
%load_ext autoreload
%autoreload 2
#Imports
import pandas as pd
import numpy as np

from pathlib import Path 

import matplotlib.pyplot as plt
import seaborn as sns
import os
from pprint import pprint
from IPython.display import display_html
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import gaussian_kde

#from scipi import  interp
from numpy import  interp

from itertools import cycle
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
import tensorflow as tf
from tensorflow.random import set_seed
set_seed(42)
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
import shap

In [ ]:
img_dpi= 300

In [ ]:
from neurocrick import plotter
from neurocrick.utils import display_side_by_side

In [ ]:
from sinfo import sinfo
sinfo()

# Set the paths
 to read the saved datasets and save the plots

In [ ]:
path_res = Path(os.path.abspath("../demo_output/coloc_tab_data_aggr_mt/"))
print(path_res)
path_res.is_dir()

In [ ]:
path_plots = path_res.joinpath("plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

# Read the data

In [ ]:
#read the saved screen feats vARIABLE
screen_feats = np.loadtxt(path_res.joinpath('screen_feats.csv'), dtype = 'str', delimiter=',')
#tmp.tolist()
screen_feats= screen_feats.tolist()
screen_feats

In [ ]:
#reading the traingin and test datasets
scal_df_train = pd.read_csv(path_res.joinpath('scal_df_train.csv'))
#scal_df_train.head()
scal_df_test = pd.read_csv(path_res.joinpath('scal_df_test.csv'))

In [ ]:
scal_model_name = path_models.joinpath('mixed-plates_coloc_aggr_mitoTox_lambThresh50_pTrans-control-plate-scaling_dense-25_3-dropout.hdf5')
print(scal_model_name)
scal_model_saved = tf.keras.models.load_model(str(scal_model_name))

# Run Shapley

In [ ]:
#Shapley
df_train_normed_summary = shap.kmeans(scal_df_train[screen_feats], 25) 

In [ ]:
df_train_normed_summary.group_names

In [ ]:
explainer = shap.KernelExplainer(scal_model_saved.predict, df_train_normed_summary)

In [ ]:
shap_values = explainer.shap_values(scal_df_test[screen_feats])

### Saving the shap values

In [ ]:
#saving the shap values
filename_shapval = path_res.joinpath(f"shap_values_{os.path.basename(scal_model_name).replace('.hdf5', '.pkl')}")
print(filename_shapval)
pickle.dump(shap_values, open(filename_shapval, 'wb'))

### Generating the plots

In [ ]:
#shap.summary_plot(shap_values[0], scal_df_test[screen_feats].head(nRecs), max_display=50, show=False)
shap.summary_plot(shap_values[0], scal_df_test[screen_feats], max_display=50, show=False)

plt.savefig(path_plots.joinpath(f"shap_summary_{os.path.basename(scal_model_name)}_aggregation.png"), 
            bbox_inches='tight', dpi= img_dpi)

In [ ]:
#shap.summary_plot(shap_values[1], scal_df_test[screen_feats].head(nRecs), max_display=50, show=False)
shap.summary_plot(shap_values[1], scal_df_test[screen_feats], max_display=50, show=False)

plt.savefig(path_plots.joinpath(f"shap_summary_{os.path.basename(scal_model_name)}_mitoTox.png"), 
            bbox_inches='tight', dpi= img_dpi)

In [ ]:
classes = np.array(["Aggregation", "Mitochondrial toxicity"])

In [ ]:
shap.summary_plot(shap_values,  scal_df_test[screen_feats], plot_type='bar', max_display=50, show=False, class_names=classes)

plt.savefig(path_plots.joinpath(f"shap_featureimportance_{os.path.basename(scal_model_name)}.png"), 
            bbox_inches='tight', dpi= img_dpi)